In [1]:
%load_ext autoreload
%autoreload 2
from anndata import AnnData
import squidpy as sq
import scanpy as sc
import matplotlib.pyplot as plt
#%matplotlib widget
import cv2
import torch
from skimage import io, morphology
import skimage.feature as features
from scipy import ndimage
import time
from tqdm.notebook import tqdm
import scipy
import seaborn as sns

import skimage
from skimage.measure import regionprops, regionprops_table, approximate_polygon
from shapely.geometry import Polygon
from shapely.geometry.polygon import LinearRing
import geopandas
import scanpy as sc
import rasterio
from rasterio import features
import shapely
from shapely.geometry import Point, Polygon
import matplotlib
from matplotlib.colors import Normalize
import spatialdata as sd
import napari
import numpy as np


c:\Users\lottep\miniconda3\envs\sparrow_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lottep\miniconda3\envs\sparrow_2\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\Users\lottep\miniconda3\envs\sparrow_2\lib\site-packages\spatialdata\__init__.py:11: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [2]:
import sparrow as sp

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


## Read in the data 

In [9]:
path=r"\\wsl.localhost\lottep\mnt\cp0293\lottep\Sparrow\Segmentation_comparison_RESOLVE\sdata_objects\RESOLVE_PROTEIN_C1-1\sdata.zarr"
sdata=sd.read_zarr(path)
sdata

SpatialData object with:
├── Images
│     ├── 'Clec4f': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'Laminin': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'alpha-Laminin': SpatialImage[cyx] (1, 21440, 17152)
│     ├── 'alpha-Laminin_2': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'clahe_2': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'clec4f_thres': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'min_max_filtered_2': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'raw_image': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'tiling_correction': SpatialImage[cyx] (1, 17152, 17152)
│     ├── 'tiling_correction_clec4f': SpatialImage[cyx] (1, 17152, 17152)
│     └── 'transcript_density': SpatialImage[cyx] (1, 17152, 17152)
├── Labels
│     ├── 'segmentation_mask': SpatialImage[yx] (17152, 17152)
│     └── 'segmentation_mask_Clec4f': SpatialImage[yx] (17152, 17152)
├── Points
│     └── 'transcripts': DataFrame with shape: (2371573, 3) (2D points)
├── Shapes
│     ├── 'Clec4f_napari': Geo

In [8]:
print(path)

\\wsl.localhost\lottep\mnt\cp0293\Sparrow\Segmentation_comparison_RESOLVE\sdata_objects\RESOLVE_PROTEIN_C1-1\sdata.zarr


## Open Napari and plot the images

In [40]:
viewer = napari.Viewer() # open napari
viewer.add_image(sdata['raw_image'])  # load in image

<Image layer 'Image' at 0x197ecd20c10>

In [41]:
viewer.add_image(sdata['alpha-Laminin_2'],colormap='green',blending='additive',name='endothelial cells')  
viewer.add_image(sdata['Clec4f'],colormap='red',blending='additive',name='kupffer cells')  


<Image layer 'kupffer cells' at 0x1980c4fce80>

## Add cells to the mix

In [ ]:
def prepare_napari_plot(sdata,cutoff_shapes=0.5,shapes_layer='segmentation_mask_boundaries',output_layer='to_plot_napari'):
    '''create polygons instead of multipolygons and simplify the shapes so the plotting is faster.'''
    temp=sdata[shapes_layer].explode()
    temp['Area']=temp.area
    temp=temp.sort_values(by='Area',ascending=False)
    temp.index=temp.index.droplevel(1)
    temp = temp[~temp.index.duplicated(keep='first')]
    if shapes_layer=='segmentation_mask_boundaries':
        temp=temp.reindex(sdata.table.obs.index)
    #adata=adata[temp.percent_area>cutoff_shapes,:]
    
    
    sdata.add_shapes(
                name=output_layer,
                shapes=sd.models.ShapesModel.parse(temp
                ),
                overwrite=True,
            )
    #adata.obsm['napari_plot']=temp2[['geometry','percent_area']]
    return sdata 

In [ ]:
sdata=prepare_napari_plot(sdata,shapes_layer='segmentation_mask_boundaries_Clec4f',output_layer='Clec4f_napari')


In [9]:
def add_napari_layer(sdata,column=None,name=None,celltype=None,colormap='magma'):
    """
    Function to plot the cells into napari, colored by the correct olumn
    If you want a subset, just take a subset of the anndata before adding this. Don't use the function to change color, only to add an extra layer."""
    if celltype: 
        adata=sdata.table[sdata.table.obs['annotation_own_score_genes']==celltype,:]
        if not name:
            name=celltype
    else:
        adata=sdata.table
    coords_to_plot=[]
    to_color=None
    #adata.obsm['napari_plot']=geopandas.GeoDataFrame(adata.obsm['napari_plot'],geometry=adata.obsm['napari_plot']['geometry'])
    if column:
        if column + "_colors" in adata.uns:
                color_dict=dict(zip(adata.obs[column].cat.categories.values,adata.uns[column + "_colors"]))
                to_color=adata.obs[column].map(color_dict)
                print('colors from adata used')
    for i in sdata['to_plot_napari'].loc[adata.obs.index.values].geometry.exterior.simplify(tolerance=2):
        coords_to_plot.append(np.array(i.coords)[:,[1,0]]) 
    Columns=dict(zip(adata.var.index.values,adata.X.T))
    Columns.update(adata.obs.to_dict('list'))
    if not name:
        name=column
    if name in viewer.layers:
        viewer.layers[name].features=Columns
        if to_color:
            print('layer reused')
            viewer.layers[name].face_color=to_color.values
        if column in viewer.layers[name].features:
            print('layer reused')
            viewer.layers[name].face_color=column
            viewer.layers[name].face_colormap =colormap
    elif to_color is not None:
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=to_color.values.astype(str),opacity=0.7,edge_width=3,edge_color='white',features=Columns)
    elif column in Columns.keys():
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=column,opacity=0.7,edge_width=3,edge_color='white',features=Columns,face_colormap=colormap)
    elif column==None:
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=colormap,opacity=0.7,edge_width=3,edge_color='white',features=Columns)

In [10]:
add_napari_layer(sdata,'annotation_own_score_genes')

colors from adata used


In [42]:
add_napari_layer(sdata,celltype='KC',colormap='red')
add_napari_layer(sdata,celltype='Stellate',colormap='yellow')
add_napari_layer(sdata,celltype='Endothelial',colormap='green')
add_napari_layer(sdata,celltype='Hepatocytes',colormap='orange')

In [43]:
add_napari_layer(sdata,celltype='Hepatocytes',colormap='orange')

## Add points into the mix

In [36]:
df=sdata.points['transcripts'].compute()


In [44]:
Kc=['Clec4f']
viewer.add_points(df[df['gene'].isin(Kc)][['y','x']].values,size=20,face_color='darkred',name='KC_genes') # I have put the size on 15, if you want to zoom less to see the spots, you can


<Points layer 'KC_genes' at 0x197841d5f90>

: 

In [34]:
genes_cytplasm=['Apoc4','Hpd','Glul','Cps1','Cyp2e1']
genes_nucleus=['Cyp3a25','Pvt1','Egfr','Stox2','Ttc39c']

In [37]:
viewer.add_points(df[df['gene'].isin(genes_cytplasm)][['y','x']].values,size=20,face_color='cyan',name='genes_cytoplasm') # I have put the size on 15, if you want to zoom less to see the spots, you can


<Points layer 'genes_cytoplasm' at 0x197f04a3eb0>

In [38]:
viewer.add_points(df[df['gene'].isin(genes_nucleus)][['y','x']].values,size=20,face_color='magenta',name='genes_nucleus') # I have put the size on 15, if you want to zoom less to see the spots, you can


<Points layer 'genes_nucleus' at 0x197f04f3e20>

## Load polygons back into python

In [16]:
import rasterio
def mask_to_polygons_layer(mask: np.ndarray) -> geopandas.GeoDataFrame:
    """Returns the polygons as GeoDataFrame

    This function converts the mask to polygons.
    https://rocreguant.com/convert-a-mask-into-a-polygon-for-images-using-shapely-and-rasterio/1786/
    """

    all_polygons = []
    all_values = []

    # Extract the polygons from the mask
    for shape, value in features.shapes(
        mask.astype(np.int32),
        mask=(mask > 0),
        transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0),
    ):
        all_polygons.append(shapely.geometry.shape(shape))
        all_values.append(int(value))

    return geopandas.GeoDataFrame(dict(geometry=all_polygons), index=all_values)

In [17]:
shape=sdata['raw_image'].shape
Portal = viewer.layers['veins'].to_labels(labels_shape=shape)

In [24]:
veins_shapes=mask_to_polygons_layer(Portal)
veins_shapes= veins_shapes.reset_index()


In [20]:
import pandas as pd

In [26]:
df=pd.DataFrame(index=sdata.table.obs.index)
for i in veins_shapes.index.values:
    df[i]=sdata['segmentation_mask_boundaries'].distance(veins_shapes.loc[i].geometry)
sdata.table.obs['distance_portal']=df.min(axis=1)

c:\Users\lottep\miniconda3\envs\sparrow_2\lib\site-packages\seaborn\categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)


<Axes: xlabel='annotation_own_score_genes', ylabel='distance_portal'>

In [33]:
import seaborn as sns 
# plot mboxplot of distance to portal vein per celltype
sns.boxplot(x='annotation_own_score_genes',y='distance_portal',data=sdata.table.obs,whis=(5,90),showfliers=False)

In [25]:
veins_shapes

,index,geometry
0,18,"POLYGON ((13962.000 1316.000, 13965.000 1316.0..."
1,19,"POLYGON ((13574.000 1398.000, 13575.000 1398.0..."
2,16,"POLYGON ((13825.000 1169.000, 13829.000 1169.0..."
3,16,"POLYGON ((13506.000 1840.000, 13506.000 1841.0..."
4,17,"POLYGON ((13899.000 1513.000, 13912.000 1513.0..."
5,20,"POLYGON ((15695.000 2908.000, 15697.000 2908.0..."
6,13,"POLYGON ((11664.000 2130.000, 11667.000 2130.0..."
7,21,"POLYGON ((15203.000 3411.000, 15204.000 3411.0..."
8,14,"POLYGON ((16247.000 5161.000, 16257.000 5161.0..."
9,11,"POLYGON ((3589.000 5734.000, 3593.000 5734.000..."
